In [ ]:
import json

file = json.load(open('json_extraction_dataset_500.json', 'r'))
print(file)

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU Check
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/tinyllama-chat-bnb-4bit"

max_seq_length = 2048  # choose sequence legth
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

In [ ]:
# Preprocess the data
from datasets import Dataset

def format_prompt(example):
  prompt = f"""###<|system|> You are a helpful AI assistant.</s> ### <|user|> {example['input']}</s> ### <|assistant|> {json.dumps(example['output'])}"""
  return prompt

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text":formatted_data})

In [ ]:
# Add LoRA Adaptors
model = FastLanguageModel.get_peft_model(
    model=model,
    r=64, # LoRA rank - higher = more capacity, more memory
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none"is optimized
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False, # Rank stabilized LoRA
    loftq_config=None, #LoftQ
)

In [ ]:
# Fine-tuning the model
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for unsloth
trainer = SFTTrainer( # Supervised Fine-tuning Trainer
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    max_seq_length=max_seq_length,
    dataset_text_field="text",
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="output",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none",  # Disable wandb logging
    ),
)

In [ ]:
# Train the model
trainer_stats = trainer.train()

In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Extract the product information:\n<div class='product'><h2>iPad Air</h2><span class='price'>$1344</span><span class='category'>audio</span><span class='brand'>Dell</span></div>"}
]

# Set the chat template if it's not already set
# if tokenizer.chat_template is None:
#     tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\n' + message['content'] + '<|im_end|>\n' }}{% elif message['role'] == 'system' %}{{ '<|im_start|>system\n' + message['content'] + '<|im_end|>\n' }}{% elif message['role'] == 'assistant' %}{{ '<|im_start|>assistant\n' + message['content'] + '<|im_end|>\n' }}{% endif %}{% if loop.last and add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}{% endfor %}"


inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

In [ ]:
# Create a GGUF model file
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
  gguf_file = os.path.join("gguf_model", gguf_files[1])
  print(f"Downloading {gguf_file}...")
  files.download(gguf_file)
else:
  print("No GGUF files found in the 'gguf_model' directory.")

In [ ]:
'''
# Run this command or login using huggingface-cli
model.push_to_hub(
    "learn-abc/html-model-tinyllama-chat-bnb-4bit",
    tokenizer,
    token=userdata.get("HF_ACCESS_TOKEN"))
'''

In [ ]:
!pip install huggingface_hub -q

In [ ]:
!huggingface-cli login

In [ ]:
model.push_to_hub("learn-abc/html-model-tinyllama-chat-bnb-4bit")

In [ ]:
tokenizer.push_to_hub("learn-abc/html-model-tinyllama-chat-bnb-4bit")

In [ ]:
'''
# Run this command or use huggingface-cli
model.push_to_hub_gguf(
    "learn-abc/html-model-tinyllama-chat-bnb-4bit-gguf",
    tokenizer,
    quantization_method="q4_k_m",
    token=userdata.get("HF_ACCESS_TOKEN"))
'''

In [ ]:
model.push_to_hub_gguf("learn-abc/html-model-tinyllama-chat-bnb-4bit-gguf")

# Fine-tuned TinyLlama for JSON Extraction

This repository contains a fine-tuned version of the `unsloth/tinyllama-chat-bnb-4bit` model, specifically trained for extracting product information from HTML snippets and outputting it in a JSON format.

## Model Details

- **Base Model:** `unsloth/tinyllama-chat-bnb-4bit`
- **Fine-tuning Method:** LoRA (Low-Rank Adaptation)
- **Trained on:** A custom dataset of HTML product snippets and their corresponding JSON representations.

## Usage

This model can be used for tasks involving structured data extraction from HTML content.

### Loading the model

You can load the model and tokenizer using the `transformers` library:

In [ ]:
"""
from unsloth import FastLanguageModel
import torch
import json

model_name = "learn-abc/html-model-tinyllama-chat-bnb-4bit" # Replace with your actual repo ID
max_seq_length = 2048 # Or your chosen sequence length
dtype = None # Auto detection

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Extract the product information:\n<div class='product'><h2>iPad Air</h2><span class='price'>$1344</span><span class='category'>audio</span><span class='brand'>Dell</span></div>"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda") # Or "cpu" if not using GPU

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs)[0]
print(response)
"""

In [ ]:
!pip freeze

# Fine-tuned TinyLlama for JSON Extraction (GGUF)

This repository contains a fine-tuned version of the `unsloth/tinyllama-chat-bnb-4bit` model, specifically trained for extracting product information from HTML snippets and outputting it in a JSON format. This is the GGUF quantized version for use with tools like `llama.cpp` or other compatible inference engines.

## Model Details

- **Base Model:** `unsloth/tinyllama-chat-bnb-4bit`
- **Fine-tuning Method:** LoRA (Low-Rank Adaptation)
- **Quantization:** q4_k_m GGUF
- **Trained on:** A custom dataset of HTML product snippets and their corresponding JSON representations.

## Usage

This model can be used for tasks involving structured data extraction from HTML content using GGUF compatible software.

### Downloading and using the GGUF file

You can download the GGUF file directly from the "Files and versions" tab on this repository page.

To use this file with `llama.cpp`, you generally follow these steps:

1.  **Download `llama.cpp`:** Clone the `llama.cpp` repository and build it. Follow the instructions in the `llama.cpp` README for building on your specific platform.

# Task
Read the provided Jupyter Notebook and generate a detailed README.md file in markdown format for a GitHub repository of the fine-tuning project. The README should include sections covering the project description, setup instructions, dataset details, model fine-tuning process, usage instructions for inference, and links to the fine-tuned model and its GGUF version on the Hugging Face Hub. The notebook file is located at "notebook.ipynb".

## Analyze the notebook

### Subtask:
Go through each cell in the notebook to understand the different stages of the fine-tuning project, including dependencies, data loading and preprocessing, model loading and configuration, training process, and evaluation/testing.


**Reasoning**:
I need to examine each cell in the notebook to understand the fine-tuning process. I will look at the code and the outputs of each cell to gather information about the steps involved, including dependencies, data handling, model specifics, training setup, and testing.



## Extract key information

### Subtask:
Identify and extract key information for the README, such as the base model used, the fine-tuning method, the dataset description, hyperparameters used for training, and instructions for running the code.


**Reasoning**:
Extract the key information from the notebook analysis and store it in a dictionary.



## Structure the readme

### Subtask:
Create a clear and detailed structure for the README.md file, including sections like Project Title, Description, Setup, Dataset, Model Fine-tuning, Usage (for inference), and Hugging Face Links.


**Reasoning**:
Outline the structure of the README.md file based on the requested sections and the extracted information.



## Generate the readme content

### Subtask:
Write the content for each section of the README.md file in markdown format, incorporating the extracted information and providing clear explanations.


**Reasoning**:
Write the content for each section of the README.md file in markdown format using the extracted information from `readme_structure` and `readme_info`.



## Add hugging face links

### Subtask:
Include the links to the Hugging Face repository for both the fine-tuned model and the GGUF version.


## Format the readme

### Subtask:
Ensure the markdown is correctly formatted for readability on GitHub.


**Reasoning**:
Review the generated readme_content string to ensure correct markdown syntax and readability on GitHub.



## Present the readme

### Subtask:
Provide the complete markdown code for the README.md file in a markdown cell.


**Reasoning**:
The README content has been generated and formatted. The final step is to provide this content in a markdown cell as requested by the subtask instructions.



**Reasoning**:
The previous command failed due to a `SyntaxError` in the markdown content within the code block. The error message points to an issue with a decimal literal, likely caused by including markdown directly in a Python string without proper escaping or using raw strings. I need to correctly format the markdown content as a raw string to avoid this issue and ensure the markdown is rendered correctly in the new cell.



**Reasoning**:
The markdown content for the README has been successfully generated and displayed as a string. The subtask requires placing this content into a markdown cell. While the previous command displayed the string, it didn't create a markdown cell with the content. The next step is to create a markdown cell and put the `readme_markdown` content inside it.



**Reasoning**:
The markdown content for the README is now stored in the `readme_markdown` variable and has been successfully displayed as a string. To fulfill the subtask of providing the complete markdown code in a markdown cell, I need to create a markdown cell and place the content of `readme_markdown` inside it. The `display()` function with a markdown string as input will achieve this.



**Reasoning**:
The subtask is to provide the complete markdown code for the README.md file in a markdown cell. The `readme_markdown` variable contains the correct markdown content. The `display()` function, when given a string that starts and ends with triple backticks and the word "markdown", will render the content as a markdown cell in the notebook. This will fulfill the subtask.



**Reasoning**:
The subtask is to provide the complete markdown code for the README.md file in a markdown cell. I have the markdown content in the `readme_markdown` variable. Using `display()` with this variable will create a markdown cell with the content.



**Reasoning**:
The subtask is to provide the complete markdown code for the README.md file in a markdown cell. I have the markdown content stored in the `readme_markdown` variable. Using the `display` function on this variable will create a markdown cell in the notebook and render the content, fulfilling the subtask.

